In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.cluster import KMeans   
from sklearn import datasets
from pycaret.clustering import *
from pycaret.datasets import *


In [2]:
df=pd.read_csv('finaldataset161023.csv')
newdf=df.dropna(inplace=False)
# newdf.to_csv('finaldataset161023cleaned.csv')
keyonly=newdf[['Nsingle_q','Ndouble_q','Ndashes','Nspaces','Nbraces','badwords_count']]

clu1=setup(data=newdf, normalize=True, categorical_features=['class'],numeric_features=['Nsingle_q','Ndouble_q','Ndashes','Nspaces','Nbraces','badwords_count'],ignore_features=['method','body','path','header'])
# x=np.array(keyonly)

# wcss=[]
# K=range(1,10)
# for k in K:  
#  kmeanModel = KMeans(n_clusters=k)  
#  kmeanModel.fit(x)  
#  wcss.append(kmeanModel.inertia_) # Plotting the distortions  
 
# plt.figure(figsize=(16,8))
# plt.plot(K, wcss)
# plt.xlabel('k')
# plt.ylabel('wcss')  
# plt.title('The Elbow Method showing the optimal clusters')  
# plt.show()




,Description,Value
0,Session id,8240
1,Original data shape,"(4391, 11)"
2,Transformed data shape,"(4391, 7)"
3,Ignore features,4
4,Ordinal features,1
5,Numeric features,6
6,Categorical features,1
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


In [3]:
# kmeans_model = KMeans(n_clusters=3, random_state=32932,max_iter=300)
# kmeans_predict = kmeans_model.fit_predict(x)
kmeans=clu1.create_model('kmeans',num_clusters=2)
kmeans


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.6846,2411.0582,0.7107,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

KMeans(n_clusters=2, random_state=8240)

In [58]:
# col1=label_0.columns
# col2=label_1.columns

# plt.scatter(label_0[col1[0]],label_0[col1[1]], color='red')
# plt.scatter(label_1[col2[0]],label_1[col2[1]], color='black')
# plt.show()
# keyonly['Cluster']=kmeans_predict


In [4]:
# plt.scatter(x[kmeans_predict == 0, 0], x[kmeans_predict == 0, 1], s = 100, c = 'red')  
# plt.scatter(x[kmeans_predict == 1, 0], x[kmeans_predict == 1, 1], s = 100, c = 'blue')  
# plt.scatter(x[kmeans_predict == 2, 0], x[kmeans_predict == 2, 1], s = 100, c = 'green')# Plott
# ing the centroids of the clusters  
# plt.scatter(kmeans_model.cluster_centers_[:, 0], kmeans_model.cluster_centers_[:,1], s = 100, c = 'yellow')
# plt.show()
plot_model(kmeans)
# evaluate_model(kmeans)



In [16]:
kmeans_result=assign_model(kmeans)
kmeans_result.to_csv(r'kmeansresult.csv')

In [6]:
dff=pd.read_csv('finaldataset161023cleaned.csv')
dff.head(10)

,Unnamed: 0,method,header,body,path,Nsingle_q,Ndouble_q,Ndashes,Nbraces,Nspaces,badwords_count,class
0,0,GET,"{'Host': ' demo.testfire.net', 'Connection': '...",/,HTTP/1.1,0,0,0,2,0,0,good
1,1,GET,{'Cookie': ' JSESSIONID=6A672F0C0562B4FA031BF6...,/VHZo2EZh,HTTP/1.1,0,0,0,2,0,0,good
2,2,GET,{'Cookie': ' JSESSIONID=6A672F0C0562B4FA031BF6...,/KYqIcNEl,HTTP/1.1,0,0,0,2,0,0,good
3,3,GET,{'Cookie': ' JSESSIONID=6A672F0C0562B4FA031BF6...,/B4XTfL7l,HTTP/1.1,0,0,0,2,0,0,good
4,4,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/,HTTP/1.1,0,0,0,3,0,0,good
5,5,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/search.jsp?query=1,HTTP/1.1,0,0,0,3,0,2,good
6,6,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/index.jsp?content=business_retirement.htm,HTTP/1.1,0,0,0,3,0,2,good
7,7,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/index.jsp?content=business_deposit.htm,HTTP/1.1,0,0,0,3,0,2,good
8,8,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/index.jsp?content=inside.htm,HTTP/1.1,0,0,0,3,0,2,good
9,9,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/index.jsp,HTTP/1.1,0,0,0,3,0,2,good


In [14]:
resulttest=predict_model(kmeans,data=dff)
# resulttest.to_csv(r'resultest.csv')
resulttest

,Nsingle_q,Ndouble_q,Ndashes,Nbraces,Nspaces,badwords_count,class,Cluster
0,-0.055533,-0.488720,-0.439059,-0.567207,0.0,-0.367625,1.691720,Cluster 0
1,-0.055533,-0.488720,-0.439059,-0.567207,0.0,-0.367625,1.691720,Cluster 0
2,-0.055533,-0.488720,-0.439059,-0.567207,0.0,-0.367625,1.691720,Cluster 0
3,-0.055533,-0.488720,-0.439059,-0.567207,0.0,-0.367625,1.691720,Cluster 0
4,-0.055533,-0.488720,-0.439059,-0.295129,0.0,-0.367625,1.691720,Cluster 0
...,...,...,...,...,...,...,...,...
4386,-0.055533,-0.488720,2.152215,-0.023050,0.0,0.645153,-0.591114,Cluster 0
4387,-0.055533,2.521055,-0.439059,3.786047,0.0,4.949460,-0.591114,Cluster 1
4388,-0.055533,-0.488720,2.152215,-0.023050,0.0,0.645153,-0.591114,Cluster 0
4389,-0.055533,-0.488720,2.152215,-0.023050,0.0,0.645153,-0.591114,Cluster 0
